# Tidal gauge data

The monthly files list the date (year-month in decimal form), mean sea level value for the month, number of missing days of data in the month, and 'flag for attention'. The Fortran fixed format for reading this data is 1x,f10.4,1x,i6,1x,i2,1x,i3. The year-month decimal form is given by year + (month-0.5)/12.0 thereby centering the monthly mean essentially in the middle of the calendar month in question. The monthly mean values are in mm.

### Importing required libraries

In [ ]:
pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
pd.set_option('precision', 7)
import plotly.express as px
from tqdm import tqdm
tqdm.pandas()
from haversine import haversine

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Saving the Data
def SaveToCSV(df,path):
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    df.to_csv(f,index = False)

In [ ]:
def NearestStation(lat,long,df): 
    yo = df
    coords = (lat,long)
    yo["Dist"]= yo.Coordinates.apply(lambda x: haversine(coords,x))
    #yo.reset_index(inplace=True)
    closest = yo.loc[yo["Dist"].idxmin()]
    return(str(closest.StationID),float(closest.Dist))

In [ ]:
#function required for Tidal data time conversion
def timeConversion(x):
  Year = int(x)
  m = x % int(x)
  Month = round(12 * m + 0.5)
  Datetime = datetime(Year, int(Month), 1)
  Date = Datetime.strftime("%Y-%m-%d")
  return Date, Year,  Month

#To get the required stations

##All Stations

In [ ]:
#loading all station details
col = ['StationID', 'Latitude','Longitude','StationName', 'CoastlineCode', 'StationCode', 'QualityControlFlag']
file_list = '/content/drive/MyDrive/filelist.txt'
station = pd.read_csv(file_list,
  #  skiprows=0,
   delimiter=';',
   names = col,
   skipinitialspace=True,
   error_bad_lines=False)

station

,StationID,Latitude,Longitude,StationName,CoastlineCode,StationCode,QualityControlFlag
0,638,64.150000,-21.933333,REYKJAVIK,10,1,N
1,877,63.833333,-22.433333,GRINDAVIK,10,11,N
2,839,62.016667,-6.766667,TORSHAVN,15,11,N
3,541,78.066667,14.250000,BARENTSBURG,25,1,N
4,547,78.066667,14.250000,BARENTSBURG II (SPITSBERGEN),25,2,N
...,...,...,...,...,...,...,...
1297,1603,-62.483333,-59.633333,PUERTO SOBERANIA,999,5,N
1298,858,-64.900000,-62.866667,ALMIRANTE BROWN,999,24,N
1299,1763,-77.033333,163.183333,CAPE ROBERTS ANTARCTICA,999,70,N
1300,2029,-77.850000,166.767000,SCOTT BASE,999,80,N


In [ ]:
station.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302 entries, 0 to 1301
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   StationID           1302 non-null   int64  
 1   Latitude            1302 non-null   float64
 2   Longitude           1302 non-null   float64
 3   StationName         1302 non-null   object 
 4   CoastlineCode       1302 non-null   int64  
 5   StationCode         1302 non-null   int64  
 6   QualityControlFlag  1302 non-null   object 
dtypes: float64(2), int64(3), object(2)
memory usage: 71.3+ KB


In [ ]:
#filtering required stations
ca_stations = station[station['CoastlineCode']==823] ## california coastal code
ca_stations['Coordinates'] = list(zip(ca_stations['Latitude'], ca_stations['Longitude']))
ca_stations

,StationID,Latitude,Longitude,StationName,CoastlineCode,StationCode,QualityControlFlag,Coordinates
983,385,48.366667,-124.616667,NEAH BAY,823,1,N,"(48.366667, -124.616667)"
984,2127,48.125000,-123.440000,"PORT ANGELES, WASHINGTON",823,3,N,"(48.125, -123.44)"
985,1633,48.866667,-122.750000,CHERRY POINT,823,5,N,"(48.866667, -122.75)"
986,384,48.550000,-123.000000,FRIDAY HARBOR (OCEAN. LABS.),823,6,N,"(48.55, -123.0)"
987,1354,46.716667,-123.966667,"TOKE POINT, WILLIPA BAY, WA",823,9,N,"(46.716667, -123.966667)"
988,127,47.600000,-122.333333,SEATTLE,823,11,N,"(47.6, -122.333333)"
989,1325,48.116667,-122.750000,PORT TOWNSEND,823,12,N,"(48.116667, -122.75)"
990,265,46.216667,-123.766667,ASTORIA (TONGUE POINT),823,13,N,"(46.216667, -123.766667)"
991,1285,45.566667,-123.916667,GARIBALDI,823,14,N,"(45.566667, -123.916667)"
992,1541,44.816667,-124.066667,DEPOE BAY,823,15,N,"(44.816667, -124.066667)"


In [ ]:
len(ca_stations)

31

## Common Coordinates

In [ ]:
Counties = ['SantaBarbara','SanDiego']
directory = '/content/drive/Shareddrives/MSDA-SLR Project/SLR Project/Data/LiDAR_Presampled_Data/'

CommonCoordinates = pd.DataFrame()
for County in Counties:
  #Common Coordinates
  CommonCoordinates_file_path = directory+'/'+County+'CommonCoordinates.csv'
  temp_df = pd.read_csv(CommonCoordinates_file_path)
  temp_df['CommonCoordinates'] = list(zip(temp_df['Latitude'], temp_df['Longitude']))
  temp_df['County'] = County
  CommonCoordinates = CommonCoordinates.append(temp_df)

CommonCoordinates

,Latitude,Longitude,CommonCoordinates,County
0,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara
1,34.4335,-119.9482,"(34.4335, -119.9482)",SantaBarbara
2,34.4336,-119.9482,"(34.4336, -119.9482)",SantaBarbara
3,34.4337,-119.9482,"(34.4337, -119.9482)",SantaBarbara
4,34.4338,-119.9482,"(34.4338, -119.9482)",SantaBarbara
...,...,...,...,...
1790,32.5386,-117.1249,"(32.5386, -117.1249)",SanDiego
1791,32.5404,-117.1249,"(32.5404, -117.1249)",SanDiego
1792,32.5366,-117.1248,"(32.5366, -117.1248)",SanDiego
1793,32.5357,-117.1247,"(32.5357, -117.1247)",SanDiego


##Flitering Relevant Stations

In [ ]:
rlvnt_stations = ca_stations[ca_stations[["Latitude","Longitude"]].round(decimals = 1).apply(tuple, 1).isin( \
                                  CommonCoordinates[["Latitude","Longitude"]].round(decimals = 1).apply(tuple, 1))]
print(rlvnt_stations.shape)
rlvnt_stations.head()

(2, 8)


,StationID,Latitude,Longitude,StationName,CoastlineCode,StationCode,QualityControlFlag,Coordinates
1006,2126,34.408333,-119.685000,"SANTA BARBARA, CALIFORNIA",823,48,N,"(34.408333, -119.685)"
1013,158,32.716667,-117.166667,SAN DIEGO (QUARANTINE STATION),823,81,N,"(32.716667, -117.166667)"


##Closest Stations

In [ ]:
CommonCoordinates['ClosestStationID'], CommonCoordinates['Distance'] = \
                            zip(*CommonCoordinates.progress_apply(
                                lambda x: NearestStation(float(x['Latitude']),float(x['Longitude']),rlvnt_stations), axis=1))
CommonCoordinates

100%|██████████| 40536/40536 [00:36<00:00, 1104.46it/s]


,Latitude,Longitude,CommonCoordinates,County,ClosestStationID,Distance
0,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara,2126,24.3000511
1,34.4335,-119.9482,"(34.4335, -119.9482)",SantaBarbara,2126,24.3038265
2,34.4336,-119.9482,"(34.4336, -119.9482)",SantaBarbara,2126,24.3050951
3,34.4337,-119.9482,"(34.4337, -119.9482)",SantaBarbara,2126,24.3063686
4,34.4338,-119.9482,"(34.4338, -119.9482)",SantaBarbara,2126,24.3076471
...,...,...,...,...,...,...
1790,32.5386,-117.1249,"(32.5386, -117.1249)",SanDiego,158,20.1828091
1791,32.5404,-117.1249,"(32.5404, -117.1249)",SanDiego,158,19.9864825
1792,32.5366,-117.1248,"(32.5366, -117.1248)",SanDiego,158,20.4028345
1793,32.5357,-117.1247,"(32.5357, -117.1247)",SanDiego,158,20.5028508


In [ ]:
CommonCoordinates.Distance.max()

29.78565918189974

In [ ]:
CommonCoordinates.Distance.min()

0.009880588293104366

In [ ]:
ClosestStationIDs = list(CommonCoordinates.ClosestStationID.unique())
ClosestStationIDs

['2126', '158']

In [ ]:
CommonCoordinates = CommonCoordinates.rename(columns={'ClosestStationID':'StationID'}) 

#Extracting Data

In [ ]:
#extracting tidal data from all the required stations
colnames = ['Timestamp', 'MeanSL', 'MissingDays', 'Flag']
required_stations_list = ClosestStationIDs
tidal_gauge_df=pd.DataFrame()
for station in required_stations_list:
  rlr_data_sd = 'https://www.psmsl.org/data/obtaining/rlr.monthly.data/' + str(station) + '.rlrdata' 
  temp_df = pd.read_csv(rlr_data_sd,header=None, sep = ';', names=colnames)
  temp_df['StationID'] = station
  temp_df[['Date','Year','Month']] = temp_df.apply(lambda x : timeConversion(x['Timestamp']),axis=1, result_type ='expand')
  temp_df = temp_df.drop(['Flag'],axis=1)
  tidal_gauge_df = tidal_gauge_df.append(temp_df)

tidal_gauge_df

,Timestamp,MeanSL,MissingDays,StationID,Date,Year,Month
0,1973.9583,7891,99,2126,1973-12-01,1973,12
1,1974.0417,-99999,0,2126,1974-01-01,1974,1
2,1974.1250,7854,0,2126,1974-02-01,1974,2
3,1974.2083,7873,0,2126,1974-03-01,1974,3
4,1974.2917,7854,0,2126,1974-04-01,1974,4
...,...,...,...,...,...,...,...
1387,2021.6250,7239,0,158,2021-08-01,2021,8
1388,2021.7083,7224,0,158,2021-09-01,2021,9
1389,2021.7917,7209,0,158,2021-10-01,2021,10
1390,2021.8750,7150,0,158,2021-11-01,2021,11


In [ ]:
tidal_gauge_df = tidal_gauge_df[tidal_gauge_df.Year>=2009]
tidal_gauge_df = tidal_gauge_df[tidal_gauge_df.Year<=2020]
tidal_gauge_df

,Timestamp,MeanSL,MissingDays,StationID,Date,Year,Month
421,2009.0417,7864,0,2126,2009-01-01,2009,1
422,2009.1250,7852,0,2126,2009-02-01,2009,2
423,2009.2083,7851,0,2126,2009-03-01,2009,3
424,2009.2917,7832,0,2126,2009-04-01,2009,4
425,2009.3750,7905,0,2126,2009-05-01,2009,5
...,...,...,...,...,...,...,...
1375,2020.6250,7239,0,158,2020-08-01,2020,8
1376,2020.7083,7247,0,158,2020-09-01,2020,9
1377,2020.7917,7223,0,158,2020-10-01,2020,10
1378,2020.8750,7147,0,158,2020-11-01,2020,11


In [ ]:
len(tidal_gauge_df.Date.unique())

144

#Preparing for merge with other data

In [ ]:
tidal_gauge_df['date']=pd.to_datetime(tidal_gauge_df['Date'])
tidal_gauge_df = tidal_gauge_df.drop('Date',axis=1)

In [ ]:
tidal_gauge_df = tidal_gauge_df.drop(['Year',	'Month','Timestamp'],axis=1)
tidal_gauge_df

,MeanSL,MissingDays,StationID,date
421,7864,0,2126,2009-01-01
422,7852,0,2126,2009-02-01
423,7851,0,2126,2009-03-01
424,7832,0,2126,2009-04-01
425,7905,0,2126,2009-05-01
...,...,...,...,...
1375,7239,0,158,2020-08-01
1376,7247,0,158,2020-09-01
1377,7223,0,158,2020-10-01
1378,7147,0,158,2020-11-01


##Preparing a DataFrame with all month years and stations

In [ ]:
temp_tg_raw_df = pd.DataFrame({'date':pd.date_range(start='2009-01-01', end='2020-12-01', freq='MS')})
temp_tg_raw_df = (pd.concat([temp_tg_raw_df] * len(ClosestStationIDs), keys=ClosestStationIDs)
   .reset_index(level=1, drop=True)
   .rename_axis('StationID')
   .reset_index()
)
temp_tg_raw_df

,StationID,date
0,2126,2009-01-01
1,2126,2009-02-01
2,2126,2009-03-01
3,2126,2009-04-01
4,2126,2009-05-01
...,...,...
283,158,2020-08-01
284,158,2020-09-01
285,158,2020-10-01
286,158,2020-11-01


In [ ]:
bckp = tidal_gauge_df
tidal_gauge_df = pd.merge(temp_tg_raw_df,tidal_gauge_df,on=['date','StationID'],how='left')
# temp_tg_raw_df.avgtemp = temp_tg_raw_df.groupby(['date','stationid'])['avgtemp'].apply(lambda x: x.fillna(x.mean()))
# temp_tg_raw_df.avgtemp = temp_tg_raw_df.avgtemp.fillna(temp_tg_raw_df.avgtemp.mean())
tidal_gauge_df

,StationID,date,MeanSL,MissingDays
0,2126,2009-01-01,7864,0
1,2126,2009-02-01,7852,0
2,2126,2009-03-01,7851,0
3,2126,2009-04-01,7832,0
4,2126,2009-05-01,7905,0
...,...,...,...,...
283,158,2020-08-01,7239,0
284,158,2020-09-01,7247,0
285,158,2020-10-01,7223,0
286,158,2020-11-01,7147,0


In [ ]:
SaveToCSV(bckp,path = '/content/drive/Shareddrives/MSDA-SLR Project/SLR Project/Data/TG_Raw.csv')

In [ ]:
#getting coordinates from actual data for merging
tg_Temp_Df = pd.merge(CommonCoordinates,tidal_gauge_df,on=['StationID'],how='left')
tg_Temp_Df

,Latitude,Longitude,CommonCoordinates,County,StationID,Distance,date,MeanSL,MissingDays
0,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara,2126,24.3000511,2009-01-01,7864,0
1,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara,2126,24.3000511,2009-02-01,7852,0
2,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara,2126,24.3000511,2009-03-01,7851,0
3,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara,2126,24.3000511,2009-04-01,7832,0
4,34.4332,-119.9482,"(34.4332, -119.9482)",SantaBarbara,2126,24.3000511,2009-05-01,7905,0
...,...,...,...,...,...,...,...,...,...
5837179,32.5381,-117.1247,"(32.5381, -117.1247)",SanDiego,158,20.2409859,2020-08-01,7239,0
5837180,32.5381,-117.1247,"(32.5381, -117.1247)",SanDiego,158,20.2409859,2020-09-01,7247,0
5837181,32.5381,-117.1247,"(32.5381, -117.1247)",SanDiego,158,20.2409859,2020-10-01,7223,0
5837182,32.5381,-117.1247,"(32.5381, -117.1247)",SanDiego,158,20.2409859,2020-11-01,7147,0


In [ ]:
tg_Temp_Df = tg_Temp_Df.drop(['Distance'],axis = 1)

In [ ]:
len(tg_Temp_Df.date.unique())

144

In [ ]:
#checking for -99999
len(tg_Temp_Df[tg_Temp_Df.MeanSL == -99999])

77482

In [ ]:
#Data description before missing value fix
tg_Temp_Df.describe()['MeanSL']

count    5.8371840e+06
mean     6.5104881e+03
std      1.2354787e+04
min     -9.9999000e+04
25%      7.9200000e+03
50%      7.9740000e+03
75%      8.0230000e+03
max      8.2010000e+03
Name: MeanSL, dtype: float64

In [ ]:
#replacing all -99999 values with Nan
tg_Temp_Df['MeanSL'].replace(-99999, np.nan, inplace=True)
#filling Nan values with mean values of each year for the corresponding stations
tg_Temp_Df.fillna(tg_Temp_Df.groupby([tg_Temp_Df.date.dt.year,'StationID']).transform('mean'), inplace=True)
#Data description after missing value fix
tg_Temp_Df.describe()['MeanSL']

count    5.8371840e+06
mean     7.9433170e+03
std      1.8080634e+02
min      7.0210000e+03
25%      7.9210000e+03
50%      7.9740000e+03
75%      8.0230000e+03
max      8.2010000e+03
Name: MeanSL, dtype: float64

In [ ]:
SaveToCSV(tg_Temp_Df,path = '/content/drive/Shareddrives/MSDA-SLR Project/SLR Project/Data/TG_SB_SD_AllCoordinates_09_20s.csv')